In [1]:
import sys

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import seaborn as sns # library to handle whatever

# libraries to perform web scraping
from urllib.request import urlopen
# pip install BeautifulSoup4
from bs4 import BeautifulSoup

import json # library to handle JSON files

!conda install --yes --prefix {sys.prefix} -c conda-forge geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install --yes --prefix {sys.prefix} -c conda-forge folium=0.5.0
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... 
  - anaconda/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.

Solving environment: ...working... 
  - anaconda/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.

Libraries imported.


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url)

In [3]:
soup = BeautifulSoup(html, 'html.parser')
#type(soup)

In [4]:
row_td = soup.find_all('td')
#print(row_td[:5])

In [5]:
for row in row_td[:-33]:
    str_cells = str(row)

#print(str_cells)

In [6]:
str_cells = str(row_td[:-33])
cleantext = BeautifulSoup(str_cells, "html.parser").get_text()
#print(cleantext[:50])

In [7]:
import re

list_rows = []
for row in row_td[:-33]:
    str_cells = str(row)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '', str_cells))
    list_rows.append(clean2)
#print(clean2)
#type(clean2)

In [22]:
df = pd.DataFrame(list_rows)
#df.head()

In [23]:
ar_df = np.array(df)
ar_df.resize(int(len(ar_df)/3), 3)
df1 = pd.DataFrame(ar_df)
#df1.head()

In [24]:
col_labels = soup.find_all('th')
#col_labels

In [25]:
all_header = []
col_str = str(col_labels)
cleantext2 = BeautifulSoup(col_str, "html.parser").get_text()
all_header.append(cleantext2)
#print(all_header)

In [26]:
df2 = pd.DataFrame(all_header)
#df2.head()

In [27]:
df3 = df2[0].str.split(',', expand=True)
#df3.head()

In [28]:
frames = [df3, df1]

df4 = pd.concat(frames)
#df4.head(10)

In [29]:
df5 = df4.rename(columns=df4.iloc[0])
#df5.head()

In [30]:
df6 = df5.replace(r'\n','', regex=True) 
#df6.head(10)

In [31]:
df7 = df6.drop(df6.index[0])
df7 = df7.drop(columns=' Canadian postal codes\n]')
#df7.head()

In [32]:
df7.rename(columns={'[Postal Code\n': 'PostalCode'},inplace=True)
df7.rename(columns={' Borough\n': 'Borough'},inplace=True)
df7.rename(columns={' Neighborhood\n': 'Neighborhood'},inplace=True)
#df7.head()

In [49]:
df8 = df7[df7.Borough != 'Not assigned']
df8 = df8.reset_index()
del df8['index']

df8.shape

(103, 3)

In [50]:
df9 = df8.sort_values(by='PostalCode')

df9 = df9.reset_index()
df9 = df9.drop(columns='index')

In [51]:
df10 = pd.read_csv('https://cocl.us/Geospatial_data')
df11 = pd.concat([df9, df10], axis=1)

In [60]:
neighborhoods = df11.drop(columns='Postal Code')
#neighborhoods

In [61]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [70]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto